In [1]:
import re

In [5]:
pipeline_fname = 'models/detection/ssd_efficientdet_d1_640x640_coco17_tpu-8.config'
fine_tune_checkpoint = "models/detection/download/efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0"
train_record_fname = "data/oxford-pet/pet_faces_train.record-?????-of-00010"
test_record_fname = "data/oxford-pet/pet_faces_val.record-?????-of-00010"
label_map_pbtxt_fname = "/Data/leo/models/research/object_detection/data/pet_label_map.pbtxt"
batch_size = 2
num_steps = 10000
num_classes = 37

with open(pipeline_fname) as f:
    s = f.read()
    
with open('models/detection/pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

```
python /Data/leo/models/research/object_detection/model_main_tf2.py \
--model_dir="model/detection/fine-tuned" --num_train_steps=100 \
--sample_1_of_n_eval_examples=10 \
--pipeline_config_path=models/detection/pipeline_file.config \
--alsologtostderr
```

In [23]:
import os
import sklearn
import PIL.Image as Image

In [18]:
dataset_path = "data/oxford-pet"
file_list_path = os.path.join(dataset_path, 'annotations', "list.txt")
with open(file_list_path) as f:
    file_list = []
    for i in range(6):
        f.readline()
    for line in f:
        f_name, cat_dog, *_ = line.split()
        file_list.append((f_name, cat_dog))
        
train, test = sklearn.model_selection.train_test_split(file_list)

In [24]:
def load_example(info, augmentation=False):
    f_name, cat_dog = info
    im_path = os.path.join(dataset_path, 'images', f_name + '.jpg')
    im = Image.open(im_path)
    return im

In [28]:
sizes = set(load_example(f).size for f in file_list)

In [29]:
sizes

{(346, 500),
 (500, 319),
 (400, 291),
 (202, 300),
 (206, 300),
 (380, 500),
 (331, 500),
 (500, 293),
 (300, 202),
 (300, 279),
 (356, 500),
 (480, 360),
 (307, 500),
 (1280, 800),
 (500, 451),
 (300, 224),
 (478, 500),
 (324, 306),
 (500, 340),
 (400, 328),
 (299, 448),
 (500, 489),
 (450, 483),
 (184, 300),
 (500, 370),
 (230, 240),
 (500, 407),
 (200, 172),
 (500, 280),
 (400, 253),
 (500, 445),
 (400, 417),
 (322, 480),
 (311, 320),
 (500, 262),
 (400, 282),
 (405, 402),
 (600, 382),
 (500, 300),
 (379, 500),
 (600, 901),
 (277, 200),
 (343, 500),
 (254, 380),
 (300, 223),
 (300, 292),
 (294, 500),
 (500, 458),
 (500, 339),
 (248, 300),
 (319, 500),
 (225, 225),
 (498, 500),
 (449, 500),
 (300, 245),
 (500, 496),
 (467, 350),
 (399, 428),
 (500, 377),
 (474, 500),
 (425, 500),
 (288, 500),
 (473, 568),
 (413, 500),
 (288, 216),
 (500, 414),
 (373, 451),
 (500, 359),
 (389, 500),
 (412, 480),
 (400, 244),
 (500, 388),
 (210, 500),
 (250, 175),
 (600, 424),
 (600, 561),
 (500, 418)